In [2]:
import pandas as pd

In [27]:
data = pd.read_csv("metadata.csv")

/home/selo/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (1,4,5,6,13,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [33]:
dir(data)
data["title"][0]
data["abstract"][14]
i = 0
for a in data["abstract"]:
    if type(a).__name__=="float":
        i += 1
        #print(a, "!", i)
print("{}/{} articles don't have abstract: Corresponds to {}%".format(i, len(data), i/len(data)*100))

54865/192509 articles don't have abstract: Corresponds to 28.499966235344843%


In [60]:
data = data.head(10)

In [ ]:
"""
1- Tokenization/stemming
2- Normalize
3- Build vocabulary after stemming
4- Collect TF-IDF
5- Implement the cosine similarity system mentioned in lectures



"""

In [24]:
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import numpy as np
from collections import Counter
from nltk.stem import WordNetLemmatizer

In [29]:
text = data["abstract"][2]
words= [word for word in word_tokenize(text) if word.isalnum()]
N = len(words)
tf = np.zeros((N,))
idf = np.zeros((N,))
counter = Counter(words)
porter = PorterStemmer()
stems = [porter.stem(word) for word in words]
lanc = LancasterStemmer()
wordnet_lemmatizer = WordNetLemmatizer()
stems2 = [wordnet_lemmatizer.lemmatize(word) for word in words]
#print(stems2)
#print("-"*40)
#print(stems)
#print("-"*40)
#print(words)

In [69]:
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import numpy as np
from collections import Counter
from nltk.stem import WordNetLemmatizer
from scipy import spatial
import scipy
from sortedcontainers import SortedList
from sklearn.metrics.pairwise import cosine_similarity

class Document:
    def __init__(self, tokens, word2id=None, doc_id=None, 
                 cord_uid=None, abstract=None, title=None):
        self.tokens = tokens
        self.doc_id = doc_id
        self.cord_uid = cord_uid
        self.abstract = abstract
        self.title = title
        self.tf_dict = Counter(tokens)
        if word2id:
            self.tf_vec = self.get_tf_vector_(word2id)
            
    def get_tf_vector_(self, word2id):
        tokens = self.tokens
        V = len(word2id) + 1
        tf_vec = [0]*V
        for token in tokens:
            if token in word2id:
                tf_vec[word2id[token]] += 1
        # store as sparse matrix
        tf_vec = scipy.sparse.csr_matrix(tf_vec)
        tf_vec[tf_vec>0] = np.log10(tf_vec[tf_vec>0]) + 1
        return tf_vec

class TrecIR:
    def __init__(self, corpus, process_type="stem"):
        self.corpus = corpus
        self.process_type = process_type
        if process_type == "stem":
            self.porter = PorterStemmer()
            self.proc_token = lambda x: self.porter.stem(x)
        elif process_type == "lemmatize":
            self.lemmatizer = WordNetLemmatizer()
            self.proc_token = lambda x: self.lemmatizer.lemmatize(x)
        else:
            raise ValueError
        
        self.process_corpus_()
        
  
    def get_ranked_docs(self, query):
        # todo: stopword elim to speed up
        query_doc = self.process_text_(query)
        
        # idf: shape: (V, 1)
        # doc_tf: (1, V)
        # q_tf_idf: (V, 1)
        q_tf_idf = self.idf.multiply(query_doc.tf_vec.T)
        ranked_docs = SortedList()
        doc_ids = set.union(*[self.posting_list[word] 
                                  for word in query_doc.tokens])
        for doc_id in doc_ids:
            doc = self.docs[doc_id]
            doc_tf_idf = self.tf_idf[doc_id]
            sim = cosine_similarity(q_tf_idf.T, doc_tf_idf)
            ranked_docs.add((-sim, doc))
        
        return [tup[1] for tup in ranked_docs]
    
    def tokenize_(self, text):
        return [word for word in word_tokenize(text) 
                      if word.isalnum()]
    
    def process_(self, tokens):
        return [self.proc_token(t) for t in tokens]
        
    def process_text_(self, text):
        tokenized_text = self.tokenize_(text)
        tokenized_text = self.process_(tokenized_text)
        return Document(tokenized_text, self.word2id)
        
    def process_corpus_(self):
        self.docs = []
        # 1 for UNK
        self.idf = dict({0: 1})
        self.word2id = dict()
        self.id2word = dict()
        self.cord2id = dict()
        self.id2cord = dict()
        self.posting_list = dict()
        # posting_list: key: string, value: set
        word_index = 1
        doc_index = 0
        for i in range(len(self.corpus)):
            if i % 10 == 1:
                print("{}%".format(i/len(self.corpus)*100), flush=True)
                
            cord_uid = data["cord_uid"][0]
            title = self.corpus["title"][i]
            abstract = self.corpus["abstract"][i]
            if type(title).__name__=="float" or type(abstract).__name__ == "float":
                # ABSTRACT DOES NOT EXIST: SKIP FOR NOW
                continue
            text= title + " " + abstract
            if cord_uid not in self.cord2id:
                self.cord2id[cord_uid] = doc_index
                self.id2cord[doc_index] = cord_uid
                doc_index += 1
            
            tokenized_text = self.tokenize_(text)
            tokenized_text = self.process_(tokenized_text)
            for word in tokenized_text:
                # add word to dictionary
                if word not in self.word2id:
                    self.word2id[word] = word_index
                    self.id2word[word_index] = word
                    word_index += 1
                # add doc to posting_list
                if word not in self.posting_list:
                    self.posting_list[word] = set()
                self.posting_list[word].add(doc_index)
            doc = Document(tokenized_text, word2id=None, 
                           doc_id=self.cord2id[cord_uid], 
                           cord_uid=cord_uid, title=title, 
                           abstract=abstract)
            self.docs.append(doc)
            for word, tf in doc.tf_dict.items():
                index = self.word2id[word]
                if index not in self.idf: 
                    self.idf[index] = 0
                self.idf[index] += 1
        
        # idf: (V, 1);
        for doc in self.docs:
            doc.tf_vec = doc.get_tf_vector_(self.word2id)
        N = len(self.corpus)
        self.idf = np.array(list(zip(*sorted(self.idf.items())))[1])
        self.idf = scipy.sparse.csr_matrix(np.log10(N/self.idf)).T
        # tf: (N, V)
        self.doc_tf = scipy.sparse.vstack([doc.tf_vec for doc in self.docs])
        # tf_idf: (N, V)
        self.tf_idf = (self.idf.multiply(self.doc_tf.T)).T
        

              

In [70]:
trec_ir = TrecIR(data)
query = "a question..." # this if for test
#qeuries = get_queries() # TODO: use the topic doc to get queries
docs = trec_ir.get_ranked_docs(query) #TODO
for doc in docs:
    print(doc.cord_uid)

10.0%
ug7v899j
